In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

N = scipy.stats.norm.cdf
n = scipy.stats.norm.pdf

class GBM():
    
    def __init__(self, sigma, mu, S0, T=2, dt=0.01):
        
        self.sigma = sigma
        self.mu = mu
        self.S0 = S0
        self.T = T
        self.dt = dt 
        
        norm = scipy.stats.norm.rvs(0, np.sqrt(dt), size = int(round(T/dt))) 
        Wt = np.cumsum(norm) #brownian motion
        
        self.trajectory = np.concatenate([np.array([self.S0]), self.S0*np.exp((mu-(self.sigma**2)/2)*np.arange(dt, T, step=dt) + self.sigma*Wt[:-1])])
                
    def plot(self):
        
        plt.plot(np.arange(self.T, step = self.dt), self.trajectory)
        plt.xlabel("t")
        plt.ylabel("S(t)")
        plt.show()
        
                      
class Option():
    
    def __init__(self, K, brownian_motion, option_type):
        
        self.K = K
        self.brownian_motion = brownian_motion  
        self.r = self.brownian_motion.mu 
        self.option_type = option_type
        
    def d1(self, t):
        return (np.log(self.brownian_motion.trajectory[int(round(t/self.brownian_motion.dt))]/self.K) + (self.r + (self.brownian_motion.sigma)**2/2)*(self.brownian_motion.T - t)) / (self.brownian_motion.sigma*np.sqrt(self.brownian_motion.T - t))
     
    def d2(self, t):
        return self.d1(t) - self.brownian_motion.sigma * np.sqrt(self.brownian_motion.T - t)       
        
    def price_option(self, t=0):
        
        if self.option_type == "call":
            return self.brownian_motion.trajectory[int(round(t/self.brownian_motion.dt))] * N(self.d1(t)) - self.K * np.exp(-self.r*(self.brownian_motion.T-t))* N(self.d2(t))
        
        elif self.option_type == "put":
            return self.K*np.exp(-self.r*(self.brownian_motion.T-t))*N(-self.d2(t)) - self.brownian_motion.trajectory[int(round(t/self.brownian_motion.dt))]*N(-self.d1(t))
    
        elif self.option_type == "call lookback":
            return self.K*N(self.d1)

    def delta(self, t=0):
        
        if self.option_type == "call":
            return N(self.d1(t))
        
        elif self.option_type == "put":
            return N(self.d1(t)) - 1
        
    def gamma(self, t=0):
        
        return n(self.d1(t))/(self.brownian_motion.trajectory[int(round(t/self.brownian_motion.dt))]*self.brownian_motion.sigma*np.sqrt(self.brownian_motion.T - t))
    
    def payoff(self):
        
        if self.option_type == "call":
            return np.max([self.brownian_motion.trajectory[-1] - self.K, 0])
        
        elif self.option_type == "put":
            return np.max([self.K - self.brownian_motion.trajectory[-1], 0])